## 1. Setup Database with Multiple Tables

In [24]:
import sqlite3
import pandas as pd

print("=" * 60)
print("SQL JOINS - COMBINING TABLES")
print("=" * 60)

# Create in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create students table
cursor.execute("""
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    course_id INTEGER
)
""")

# Create courses table
cursor.execute("""
CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT,
    instructor TEXT
)
""")

# Insert students data
students = [
    (1, 'Alice', 101),
    (2, 'Bob', 102),
    (3, 'Carol', 101),
    (4, 'David', 103),
    (5, 'Eve', None)  # No course assigned
]

# Insert courses data
courses = [
    (101, 'Python', 'Dr. Smith'),
    (102, 'SQL', 'Dr. Johnson'),
    (103, 'JavaScript', 'Dr. Lee'),
    (104, 'Java', 'Dr. Brown')  # No students enrolled
]

cursor.executemany("INSERT INTO students VALUES (?, ?, ?)", students)
cursor.executemany("INSERT INTO courses VALUES (?, ?, ?)", courses)
conn.commit()

print("✓ Database setup complete")
print(f"  - {len(students)} students created")
print(f"  - {len(courses)} courses created")

SQL JOINS - COMBINING TABLES
✓ Database setup complete
  - 5 students created
  - 4 courses created


## 1. INNER JOIN

**INNER JOIN** returns only rows where the join condition matches in BOTH tables.

Use INNER JOIN when:
- You need matching data only
- You want to exclude non-matching records
- Example: Students who have enrolled in courses

In [25]:
print("\n" + "=" * 60)
print("1. INNER JOIN")
print("=" * 60)

print("\nConcept: Only matching records from both tables")
print("\nSQL Syntax:")
print("SELECT s.name, c.course_name")
print("FROM students s")
print("INNER JOIN courses c ON s.course_id = c.course_id;")
print("\nResult:")

query = """
SELECT s.student_id, s.name, s.course_id, c.course_name, c.instructor
FROM students s
INNER JOIN courses c ON s.course_id = c.course_id
"""

result = pd.read_sql_query(query, conn)
print(result)
print(f"\n✓ {len(result)} rows (Eve and Java course excluded - no match)")


1. INNER JOIN

Concept: Only matching records from both tables

SQL Syntax:
SELECT s.name, c.course_name
FROM students s
INNER JOIN courses c ON s.course_id = c.course_id;

Result:
   student_id   name  course_id course_name   instructor
0           1  Alice        101      Python    Dr. Smith
1           2    Bob        102         SQL  Dr. Johnson
2           3  Carol        101      Python    Dr. Smith
3           4  David        103  JavaScript      Dr. Lee

✓ 4 rows (Eve and Java course excluded - no match)


## 2. LEFT JOIN (LEFT OUTER JOIN)

**LEFT JOIN** returns ALL rows from the left table, plus matching rows from the right table.

Use LEFT JOIN when:
- You need all records from the left table
- You want to include non-matching records
- Example: All students, showing their courses if enrolled

In [26]:
print("\n" + "=" * 60)
print("2. LEFT JOIN")
print("=" * 60)

print("\nConcept: ALL rows from left table + matching from right")
print("NULLs appear for non-matching right table values")
print("\nSQL Syntax:")
print("SELECT s.name, c.course_name")
print("FROM students s")
print("LEFT JOIN courses c ON s.course_id = c.course_id;")
print("\nResult:")

query = """
SELECT s.student_id, s.name, s.course_id, c.course_name, c.instructor
FROM students s
LEFT JOIN courses c ON s.course_id = c.course_id
"""

result = pd.read_sql_query(query, conn)
print(result)
print(f"\n✓ {len(result)} rows (Eve included with NULL course - LEFT table preserved)")


2. LEFT JOIN

Concept: ALL rows from left table + matching from right
NULLs appear for non-matching right table values

SQL Syntax:
SELECT s.name, c.course_name
FROM students s
LEFT JOIN courses c ON s.course_id = c.course_id;

Result:
   student_id   name  course_id course_name   instructor
0           1  Alice      101.0      Python    Dr. Smith
1           2    Bob      102.0         SQL  Dr. Johnson
2           3  Carol      101.0      Python    Dr. Smith
3           4  David      103.0  JavaScript      Dr. Lee
4           5    Eve        NaN        None         None

✓ 5 rows (Eve included with NULL course - LEFT table preserved)


## 3. RIGHT JOIN (RIGHT OUTER JOIN)

**RIGHT JOIN** returns ALL rows from the right table, plus matching rows from the left table.

Note: SQLite doesn't support RIGHT JOIN directly, so we reverse the tables with LEFT JOIN instead.

In [27]:
print("\n" + "=" * 60)
print("3. RIGHT JOIN")
print("=" * 60)

print("\nConcept: ALL rows from right table + matching from left")
print("Note: SQLite doesn't support RIGHT JOIN")
print("Workaround: Use LEFT JOIN with tables reversed")
print("\nSQL Syntax:")
print("SELECT c.course_id, c.course_name, s.name")
print("FROM courses c")
print("LEFT JOIN students s ON c.course_id = s.course_id;")
print("\nResult:")

query = """
SELECT c.course_id, c.course_name, c.instructor, s.student_id, s.name
FROM courses c
LEFT JOIN students s ON c.course_id = s.course_id
ORDER BY c.course_id
"""

result = pd.read_sql_query(query, conn)
print(result)
print(f"\n✓ {len(result)} rows (Java course included with NULL student - RIGHT table preserved)")


3. RIGHT JOIN

Concept: ALL rows from right table + matching from left
Note: SQLite doesn't support RIGHT JOIN
Workaround: Use LEFT JOIN with tables reversed

SQL Syntax:
SELECT c.course_id, c.course_name, s.name
FROM courses c
LEFT JOIN students s ON c.course_id = s.course_id;

Result:
   course_id course_name   instructor  student_id   name
0        101      Python    Dr. Smith         1.0  Alice
1        101      Python    Dr. Smith         3.0  Carol
2        102         SQL  Dr. Johnson         2.0    Bob
3        103  JavaScript      Dr. Lee         4.0  David
4        104        Java    Dr. Brown         NaN   None

✓ 5 rows (Java course included with NULL student - RIGHT table preserved)


## 4. FULL OUTER JOIN

**FULL OUTER JOIN** returns ALL rows from BOTH tables, with NULLs where there's no match.

Note: SQLite doesn't support FULL OUTER JOIN directly. We use UNION of LEFT and RIGHT joins.

In [28]:
print("\n" + "=" * 60)
print("4. FULL OUTER JOIN")
print("=" * 60)

print("\nConcept: ALL rows from BOTH tables")
print("Note: SQLite doesn't support FULL OUTER JOIN")
print("Workaround: UNION of LEFT and RIGHT joins")
print("\nResult:")

query = """
SELECT s.student_id, s.name, s.course_id, c.course_id as c_id, c.course_name
FROM students s
LEFT JOIN courses c ON s.course_id = c.course_id
UNION
SELECT s.student_id, s.name, s.course_id, c.course_id, c.course_name
FROM students s
RIGHT JOIN courses c ON s.course_id = c.course_id
"""

result = pd.read_sql_query(query, conn)
print(result)
print(f"\n✓ {len(result)} rows (all students and all courses combined)")


4. FULL OUTER JOIN

Concept: ALL rows from BOTH tables
Note: SQLite doesn't support FULL OUTER JOIN
Workaround: UNION of LEFT and RIGHT joins

Result:
   student_id   name  course_id   c_id course_name
0         NaN   None        NaN  104.0        Java
1         1.0  Alice      101.0  101.0      Python
2         2.0    Bob      102.0  102.0         SQL
3         3.0  Carol      101.0  101.0      Python
4         4.0  David      103.0  103.0  JavaScript
5         5.0    Eve        NaN    NaN        None

✓ 6 rows (all students and all courses combined)


## 5. CROSS JOIN

**CROSS JOIN** returns the Cartesian product of two tables - every row from the left table combined with every row from the right table.

Result size = (left table rows) × (right table rows)

In [29]:
print("\n" + "=" * 60)
print("5. CROSS JOIN (CARTESIAN PRODUCT)")
print("=" * 60)

print("\nConcept: ALL combinations of rows from both tables")
print("No join condition needed")
print("Result size = 5 students × 4 courses = 20 rows")
print("\nSQL Syntax:")
print("SELECT s.name, c.course_name FROM students s CROSS JOIN courses c;")
print("\nResult (first 10 rows):")

query = """
SELECT s.name, c.course_name
FROM students s
CROSS JOIN courses c
LIMIT 10
"""

result = pd.read_sql_query(query, conn)
print(result)
print(f"\n✓ Total rows: {len(pd.read_sql_query('SELECT s.name, c.course_name FROM students s CROSS JOIN courses c', conn))} (5×4)")


5. CROSS JOIN (CARTESIAN PRODUCT)

Concept: ALL combinations of rows from both tables
No join condition needed
Result size = 5 students × 4 courses = 20 rows

SQL Syntax:
SELECT s.name, c.course_name FROM students s CROSS JOIN courses c;

Result (first 10 rows):
    name course_name
0  Alice      Python
1  Alice         SQL
2  Alice  JavaScript
3  Alice        Java
4    Bob      Python
5    Bob         SQL
6    Bob  JavaScript
7    Bob        Java
8  Carol      Python
9  Carol         SQL

✓ Total rows: 20 (5×4)


## 6. SELF JOIN

**SELF JOIN** joins a table with itself using different aliases. Useful for hierarchical data.

Common use cases:
- Employee-Manager relationships
- Category hierarchies
- Comment threads (reply-to relationships)

In [30]:
print("\n" + "=" * 60)
print("6. SELF-JOIN (Table with Itself)")
print("=" * 60)

# Create employee hierarchy
cursor.execute("""
DROP TABLE IF EXISTS employees
""")

cursor.execute("""
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY,
    name TEXT,
    manager_id INTEGER,
    salary REAL
)
""")

employees_data = [
    (1, 'Alice', None, 90000),    # CEO
    (2, 'Bob', 1, 75000),         # Reports to Alice
    (3, 'Carol', 1, 75000),       # Reports to Alice
    (4, 'David', 2, 60000),       # Reports to Bob
    (5, 'Eve', 2, 60000),         # Reports to Bob
    (6, 'Frank', 3, 55000)        # Reports to Carol
]

cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", employees_data)
conn.commit()

print("\nConcept: Join table with itself using aliases")
print("(e = employee, m = manager)")
print("\nSQL Syntax:")
print("SELECT e.name as Employee, m.name as Manager")
print("FROM employees e")
print("LEFT JOIN employees m ON e.manager_id = m.emp_id;")
print("\nResult:")

query = """
SELECT e.emp_id, e.name as Employee, e.salary, m.name as Manager, m.salary as Manager_Salary
FROM employees e
LEFT JOIN employees m ON e.manager_id = m.emp_id
ORDER BY e.emp_id
"""

result = pd.read_sql_query(query, conn)
print(result)
print(f"\n✓ Hierarchical structure: {len(result)} employee-manager relationships")


6. SELF-JOIN (Table with Itself)

Concept: Join table with itself using aliases
(e = employee, m = manager)

SQL Syntax:
SELECT e.name as Employee, m.name as Manager
FROM employees e
LEFT JOIN employees m ON e.manager_id = m.emp_id;

Result:
   emp_id Employee   salary Manager  Manager_Salary
0       1    Alice  90000.0    None             NaN
1       2      Bob  75000.0   Alice         90000.0
2       3    Carol  75000.0   Alice         90000.0
3       4    David  60000.0     Bob         75000.0
4       5      Eve  60000.0     Bob         75000.0
5       6    Frank  55000.0   Carol         75000.0

✓ Hierarchical structure: 6 employee-manager relationships


## Summary: Join Types Comparison

| Join Type | Rows From | Use Case |
|-----------|-----------|----------|
| **INNER** | Both match | Matching data only |
| **LEFT** | All left + matches | All from left table |
| **RIGHT** | All right + matches | All from right table |
| **FULL** | All both | All from both tables |
| **CROSS** | All combos | Every combination |
| **SELF** | Same table | Hierarchies, relationships |

### Key Points:
- Join condition in ON clause
- Use table aliases for clarity
- NULL values appear for non-matches
- Column names must be unambiguous

In [31]:
import sqlite3
import pandas as pd

print("=" * 50)
print("SQL JOINS")
print("=" * 50)

# Create in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create students table
cursor.execute("""
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    course_id INTEGER
)
""")

# Create courses table
cursor.execute("""
CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT,
    instructor TEXT
)
""")

# Insert data
students = [
    (1, 'Alice', 101),
    (2, 'Bob', 102),
    (3, 'Carol', 101),
    (4, 'David', 103),
    (5, 'Eve', None)  # No course assigned
]

courses = [
    (101, 'Python', 'Dr. Smith'),
    (102, 'SQL', 'Dr. Johnson'),
    (103, 'JavaScript', 'Dr. Lee'),
    (104, 'Java', 'Dr. Brown')  # No students
]

cursor.executemany("INSERT INTO students VALUES (?, ?, ?)", students)
cursor.executemany("INSERT INTO courses VALUES (?, ?, ?)", courses)
conn.commit()

print("✓ Database setup complete")

SQL JOINS
✓ Database setup complete


## 2. INNER JOIN

In [32]:
print("\n" + "=" * 50)
print("INNER JOIN")
print("=" * 50)

print("\nConcept: Returns rows where key exists in BOTH tables")
print("Missing data: NOT included")
print("\nSQL Syntax:")
print("SELECT s.name, c.course_name FROM students s")
print("INNER JOIN courses c ON s.course_id = c.course_id;")
print("\nResult:")

query = """
SELECT s.student_id, s.name, s.course_id, c.course_name, c.instructor
FROM students s
INNER JOIN courses c ON s.course_id = c.course_id
"""

result = pd.read_sql_query(query, conn)
print(result)

print(f"\nNote: {len(result)} rows returned (Eve and Java course excluded)")



INNER JOIN

Concept: Returns rows where key exists in BOTH tables
Missing data: NOT included

SQL Syntax:
SELECT s.name, c.course_name FROM students s
INNER JOIN courses c ON s.course_id = c.course_id;

Result:
   student_id   name  course_id course_name   instructor
0           1  Alice        101      Python    Dr. Smith
1           2    Bob        102         SQL  Dr. Johnson
2           3  Carol        101      Python    Dr. Smith
3           4  David        103  JavaScript      Dr. Lee

Note: 4 rows returned (Eve and Java course excluded)


## 3. LEFT JOIN (LEFT OUTER JOIN)

In [33]:
print("\n" + "=" * 50)
print("LEFT JOIN (LEFT OUTER JOIN)")
print("=" * 50)

print("\nConcept: Returns ALL rows from LEFT table")
print("         + matching rows from RIGHT table")
print("         NULLs for non-matching")
print("\nSQL Syntax:")
print("SELECT s.name, c.course_name FROM students s")
print("LEFT JOIN courses c ON s.course_id = c.course_id;")
print("\nResult:")

query = """
SELECT s.student_id, s.name, s.course_id, c.course_name, c.instructor
FROM students s
LEFT JOIN courses c ON s.course_id = c.course_id
"""

result = pd.read_sql_query(query, conn)
print(result)

print(f"\nNote: {len(result)} rows returned (Eve included with NULL course info)")



LEFT JOIN (LEFT OUTER JOIN)

Concept: Returns ALL rows from LEFT table
         + matching rows from RIGHT table
         NULLs for non-matching

SQL Syntax:
SELECT s.name, c.course_name FROM students s
LEFT JOIN courses c ON s.course_id = c.course_id;

Result:
   student_id   name  course_id course_name   instructor
0           1  Alice      101.0      Python    Dr. Smith
1           2    Bob      102.0         SQL  Dr. Johnson
2           3  Carol      101.0      Python    Dr. Smith
3           4  David      103.0  JavaScript      Dr. Lee
4           5    Eve        NaN        None         None

Note: 5 rows returned (Eve included with NULL course info)


## 4. RIGHT JOIN

In [34]:
print("\n" + "=" * 50)
print("RIGHT JOIN (RIGHT OUTER JOIN)")
print("=" * 50)

print("\nConcept: Returns ALL rows from RIGHT table")
print("         + matching rows from LEFT table")
print("         NULLs for non-matching")
print("\nNote: SQLite doesn't support RIGHT JOIN directly")
print("Workaround: Reverse the tables in LEFT JOIN")
print("\nEquivalent SQL:")
print("SELECT c.course_id, c.course_name, c.instructor, s.name")
print("FROM courses c")
print("LEFT JOIN students s ON c.course_id = s.course_id;")
print("\nResult:")

query = """
SELECT c.course_id, c.course_name, c.instructor, s.student_id, s.name
FROM courses c
LEFT JOIN students s ON c.course_id = s.course_id
ORDER BY c.course_id
"""

result = pd.read_sql_query(query, conn)
print(result)

print(f"\nNote: {len(result)} rows returned (Java course included with NULL student)")



RIGHT JOIN (RIGHT OUTER JOIN)

Concept: Returns ALL rows from RIGHT table
         + matching rows from LEFT table
         NULLs for non-matching

Note: SQLite doesn't support RIGHT JOIN directly
Workaround: Reverse the tables in LEFT JOIN

Equivalent SQL:
SELECT c.course_id, c.course_name, c.instructor, s.name
FROM courses c
LEFT JOIN students s ON c.course_id = s.course_id;

Result:
   course_id course_name   instructor  student_id   name
0        101      Python    Dr. Smith         1.0  Alice
1        101      Python    Dr. Smith         3.0  Carol
2        102         SQL  Dr. Johnson         2.0    Bob
3        103  JavaScript      Dr. Lee         4.0  David
4        104        Java    Dr. Brown         NaN   None

Note: 5 rows returned (Java course included with NULL student)


## 5. FULL OUTER JOIN

In [35]:
print("\n" + "=" * 50)
print("FULL OUTER JOIN")
print("=" * 50)

print("\nConcept: Returns ALL rows from BOTH tables")
print("         NULLs for non-matching on either side")
print("\nNote: SQLite doesn't support FULL OUTER JOIN directly")
print("Workaround: Use UNION of LEFT and RIGHT joins")
print("\nEquivalent SQL:")
print("SELECT * FROM students s")
print("LEFT JOIN courses c ON s.course_id = c.course_id")
print("UNION")
print("SELECT * FROM students s")
print("RIGHT JOIN courses c ON s.course_id = c.course_id;")
print("\nResult:")

query = """
SELECT s.student_id, s.name, s.course_id, c.course_id as c_id, c.course_name
FROM students s
LEFT JOIN courses c ON s.course_id = c.course_id
UNION
SELECT s.student_id, s.name, s.course_id, c.course_id, c.course_name
FROM students s
RIGHT JOIN courses c ON s.course_id = c.course_id
"""

result = pd.read_sql_query(query, conn)
print(result)

print(f"\nNote: {len(result)} rows returned (includes all students and all courses)")



FULL OUTER JOIN

Concept: Returns ALL rows from BOTH tables
         NULLs for non-matching on either side

Note: SQLite doesn't support FULL OUTER JOIN directly
Workaround: Use UNION of LEFT and RIGHT joins

Equivalent SQL:
SELECT * FROM students s
LEFT JOIN courses c ON s.course_id = c.course_id
UNION
SELECT * FROM students s
RIGHT JOIN courses c ON s.course_id = c.course_id;

Result:
   student_id   name  course_id   c_id course_name
0         NaN   None        NaN  104.0        Java
1         1.0  Alice      101.0  101.0      Python
2         2.0    Bob      102.0  102.0         SQL
3         3.0  Carol      101.0  101.0      Python
4         4.0  David      103.0  103.0  JavaScript
5         5.0    Eve        NaN    NaN        None

Note: 6 rows returned (includes all students and all courses)


## 6. CROSS JOIN

In [36]:
print("\n" + "=" * 50)
print("CROSS JOIN (CARTESIAN PRODUCT)")
print("=" * 50)

print("\nConcept: Returns combinations of ALL rows from BOTH tables")
print("         No join condition needed")
print("         Result size = Table1 rows × Table2 rows")
print("\nSQL Syntax:")
print("SELECT s.name, c.course_name FROM students s")
print("CROSS JOIN courses c;")
print("\nResult (first 10 rows of 20 total):")

query = """
SELECT s.name, c.course_name
FROM students s
CROSS JOIN courses c
LIMIT 10
"""

result = pd.read_sql_query(query, conn)
print(result)

print(f"\nTotal: 5 students × 4 courses = 20 rows")



CROSS JOIN (CARTESIAN PRODUCT)

Concept: Returns combinations of ALL rows from BOTH tables
         No join condition needed
         Result size = Table1 rows × Table2 rows

SQL Syntax:
SELECT s.name, c.course_name FROM students s
CROSS JOIN courses c;

Result (first 10 rows of 20 total):
    name course_name
0  Alice      Python
1  Alice         SQL
2  Alice  JavaScript
3  Alice        Java
4    Bob      Python
5    Bob         SQL
6    Bob  JavaScript
7    Bob        Java
8  Carol      Python
9  Carol         SQL

Total: 5 students × 4 courses = 20 rows


## 7. Self-Join

In [37]:
print("\n" + "=" * 50)
print("SELF-JOIN")
print("=" * 50)

# Create employee table with manager
cursor.execute("""
DROP TABLE IF EXISTS employees
""")

cursor.execute("""
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY,
    name TEXT,
    manager_id INTEGER,
    salary REAL
)
""")

employees_data = [
    (1, 'Alice', None, 90000),  # CEO
    (2, 'Bob', 1, 75000),
    (3, 'Carol', 1, 75000),
    (4, 'David', 2, 60000),
    (5, 'Eve', 2, 60000),
    (6, 'Frank', 3, 55000)
]

cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", employees_data)
conn.commit()

print("\nConcept: Join a table with itself")
print("         Use table aliases (e and m)")
print("\nSQL Syntax:")
print("SELECT e.name as Employee, m.name as Manager")
print("FROM employees e")
print("LEFT JOIN employees m ON e.manager_id = m.emp_id;")
print("\nResult:")

query = """
SELECT e.name as Employee, e.salary, m.name as Manager, m.salary as Manager_Salary
FROM employees e
LEFT JOIN employees m ON e.manager_id = m.emp_id
ORDER BY e.emp_id
"""

result = pd.read_sql_query(query, conn)
print(result)



SELF-JOIN

Concept: Join a table with itself
         Use table aliases (e and m)

SQL Syntax:
SELECT e.name as Employee, m.name as Manager
FROM employees e
LEFT JOIN employees m ON e.manager_id = m.emp_id;

Result:
  Employee   salary Manager  Manager_Salary
0    Alice  90000.0    None             NaN
1      Bob  75000.0   Alice         90000.0
2    Carol  75000.0   Alice         90000.0
3    David  60000.0     Bob         75000.0
4      Eve  60000.0     Bob         75000.0
5    Frank  55000.0   Carol         75000.0


## 8. Join Comparison Summary

In [38]:
print("\n" + "=" * 50)
print("JOIN TYPES COMPARISON")
print("=" * 50)

comparison = """
╔═════════════════╦══════════════╦═══════════════════════════════════╗
║ JOIN TYPE       ║ ROWS FROM    ║ DESCRIPTION                       ║
╠═════════════════╬══════════════╬═══════════════════════════════════╣
║ INNER           ║ Both match   ║ Only matching records             ║
║ LEFT OUTER      ║ All from L   ║ All left + matching right         ║
║ RIGHT OUTER     ║ All from R   ║ All right + matching left         ║
║ FULL OUTER      ║ All both     ║ All records from both tables      ║
║ CROSS           ║ All combos   ║ Cartesian product (no condition)  ║
║ SELF            ║ Same table   ║ Table joins itself (use aliases)  ║
╚═════════════════╩══════════════╩═══════════════════════════════════╝

When to Use:
- INNER JOIN: When you need matching data only
- LEFT JOIN: When you need all from left (e.g., all customers)
- RIGHT JOIN: When you need all from right (e.g., all products)
- FULL JOIN: When you need all from both tables
- CROSS JOIN: When you need all combinations
- SELF JOIN: For hierarchical relationships
"""
print(comparison)



JOIN TYPES COMPARISON

╔═════════════════╦══════════════╦═══════════════════════════════════╗
║ JOIN TYPE       ║ ROWS FROM    ║ DESCRIPTION                       ║
╠═════════════════╬══════════════╬═══════════════════════════════════╣
║ INNER           ║ Both match   ║ Only matching records             ║
║ LEFT OUTER      ║ All from L   ║ All left + matching right         ║
║ RIGHT OUTER     ║ All from R   ║ All right + matching left         ║
║ FULL OUTER      ║ All both     ║ All records from both tables      ║
║ CROSS           ║ All combos   ║ Cartesian product (no condition)  ║
║ SELF            ║ Same table   ║ Table joins itself (use aliases)  ║
╚═════════════════╩══════════════╩═══════════════════════════════════╝

When to Use:
- INNER JOIN: When you need matching data only
- LEFT JOIN: When you need all from left (e.g., all customers)
- RIGHT JOIN: When you need all from right (e.g., all products)
- FULL JOIN: When you need all from both tables
- CROSS JOIN: When you need al